# N-gram 언어 모델 생성하기

이 노트북에서는 NLTK를 사용하여 N-gram 언어 모델을 만드는 방법을 배워보겠습니다.
햄릿 텍스트를 사용하여 3-gram 모델을 훈련하고, 텍스트를 생성하고, 확률을 계산해보겠습니다.


## 1. 필요한 라이브러리 import


In [1]:
from nltk.corpus.reader import PlaintextCorpusReader
from nltk.util import everygrams
from nltk.lm.preprocessing import (
    pad_both_ends,
    flatten,
    padded_everygram_pipeline,
)
from nltk.lm import MLE


## 2. NLTK 데이터 다운로드

NLTK의 punkt 토크나이저가 필요합니다. 없으면 자동으로 다운로드합니다.


In [2]:
try:
    import nltk
    nltk.data.find("tokenizers/punkt.zip")
    print("NLTK punkt 토크나이저가 이미 설치되어 있습니다.")
except LookupError:
    print("NLTK punkt 토크나이저를 다운로드합니다...")
    nltk.download("punkt")


NLTK punkt 토크나이저를 다운로드합니다...


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\younghl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 3. 코퍼스 생성 및 데이터 확인

햄릿 텍스트 파일을 읽어서 코퍼스를 만들고 문장들을 확인해보겠습니다.


In [11]:
# 현재 디렉토리에서 .txt 파일들로 코퍼스 생성
my_corpus = PlaintextCorpusReader("../../", ".*\.txt")
file_ids = "./data/hamlet.txt"

# 전체 문장 개수 확인
sentences = list(my_corpus.sents(fileids=file_ids))
print(f"전체 문장 개수: {len(sentences)}")
print(f"첫 번째 문장: {sentences[0]}")
print(f"두 번째 문장: {sentences[1]}")


전체 문장 개수: 2660
첫 번째 문장: ['The', 'Project', 'Gutenberg', 'EBook', 'of', 'Hamlet', ',', 'by', 'William', 'Shakespeare']
두 번째 문장: ['This', 'eBook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.']


## 4. 몇 가지 문장 출력해보기

처음 10개 문장을 확인해보겠습니다.


In [12]:
# 처음 10개 문장만 출력
for i, sent in enumerate(my_corpus.sents(fileids=file_ids)):
    if i >= 10:
        break
    print(f"문장 {i+1}: {sent}")


문장 1: ['The', 'Project', 'Gutenberg', 'EBook', 'of', 'Hamlet', ',', 'by', 'William', 'Shakespeare']
문장 2: ['This', 'eBook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.']
문장 3: ['You', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're', '-', 'use', 'it', 'under', 'the', 'terms', 'of', 'the', 'Project', 'Gutenberg', 'License', 'included', 'with', 'this', 'eBook', 'or', 'online', 'at', 'www', '.', 'gutenberg', '.', 'org']
문장 4: ['Title', ':', 'Hamlet']
문장 5: ['Author', ':', 'William', 'Shakespeare']
문장 6: ['Editor', ':', 'Charles', 'Kean']
문장 7: ['Release', 'Date', ':', 'January', '10', ',', '2009', '[', 'EBook', '#', '27761', ']']
문장 8: ['Language', ':', 'English']
문장 9: ['Character', 'set', 'encoding', ':', 'UTF', '-', '8']
문장 10: ['***', 'START', 'OF', 'THIS', 'PROJECT', 'GUTENBERG', 'EBOOK', 'HAMLET', '***']


## 5. 패딩과 Everygrams 이해하기

N-gram 모델에서는 문장의 시작과 끝을 표시하기 위해 패딩을 사용합니다.
특정 문장(1104번째)을 예시로 패딩과 everygrams이 어떻게 작동하는지 보겠습니다.


In [18]:
# 1104번째 문장 확인
example_sentence = my_corpus.sents(fileids=file_ids)[1104]
print(f"원본 문장: {example_sentence}")

# 패딩 적용 (n=2는 trigram을 위해 양쪽에 2개씩 패딩)
padded_trigrams = list(pad_both_ends(example_sentence, n=2))
print(f"패딩된 문장: {padded_trigrams}")


원본 문장: ['_Ham', '.', '_', 'To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question', ':[', '8', ']', 'Whether', "'", 'tis', 'nobler', 'in', 'the', 'mind', 'to', 'suffer', 'The', 'slings', 'and', 'arrows', 'of', 'outrageous', 'fortune', ',', 'Or', 'to', 'take', 'arms', 'against', 'a', 'sea', 'of', 'troubles', ',[', '9', ']', 'And', ',', 'by', 'opposing', 'end', 'them', '?--', 'To', 'die', ',--', 'to', 'sleep', ',', 'No', 'more', ';--', 'and', 'by', 'a', 'sleep', ',', 'to', 'say', 'we', 'end', 'The', 'heart', '-', 'ache', ',', 'and', 'the', 'thousand', 'natural', 'shocks', 'That', 'flesh', 'is', 'heir', 'to', ':', "'", 'tis', 'a', 'consummation', 'Devoutly', 'to', 'be', 'wished', '.']
패딩된 문장: ['<s>', '_Ham', '.', '_', 'To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question', ':[', '8', ']', 'Whether', "'", 'tis', 'nobler', 'in', 'the', 'mind', 'to', 'suffer', 'The', 'slings', 'and', 'arrows', 'of', 'outrageous', 'fortune', ',', 'Or', 'to', 'take

In [19]:
# Everygrams 생성 (1-gram부터 3-gram까지)
everygrams_list = list(everygrams(padded_trigrams, max_len=3))
print(f"생성된 everygrams 개수: {len(everygrams_list)}")
print("\n처음 10개 everygrams:")
for i, gram in enumerate(everygrams_list[:10]):
    print(f"{i+1}: {gram}")


생성된 everygrams 개수: 294

처음 10개 everygrams:
1: ('<s>',)
2: ('<s>', '_Ham')
3: ('<s>', '_Ham', '.')
4: ('_Ham',)
5: ('_Ham', '.')
6: ('_Ham', '.', '_')
7: ('.',)
8: ('.', '_')
9: ('.', '_', 'To')
10: ('_',)


## 6. 전체 코퍼스에 패딩 적용

모든 문장에 패딩을 적용하고 평평하게 만들어봅시다.


In [30]:
# 전체 코퍼스에 패딩 적용
flattened_corpus = list(
    flatten(
        pad_both_ends(sent, n=3)
        for sent in my_corpus.sents(fileids=file_ids)
    )
)

print(f"평평하게 만든 코퍼스의 토큰 개수: {len(flattened_corpus)}")
print(f"처음 20개 토큰: {flattened_corpus[:20]}")


평평하게 만든 코퍼스의 토큰 개수: 62000
처음 20개 토큰: ['<s>', '<s>', 'The', 'Project', 'Gutenberg', 'EBook', 'of', 'Hamlet', ',', 'by', 'William', 'Shakespeare', '</s>', '</s>', '<s>', '<s>', 'This', 'eBook', 'is', 'for']


## 7. 훈련 데이터와 어휘집 생성

`padded_everygram_pipeline`을 사용하여 훈련 데이터와 어휘집을 한 번에 생성합니다.


In [38]:
# 3-gram 모델을 위한 훈련 데이터와 어휘집 생성
train, vocab = padded_everygram_pipeline(
    3, my_corpus.sents(fileids=file_ids)
)

print("훈련 데이터와 어휘집이 생성되었습니다.")
# print(f"훈련 데이터 크기: {len(list(train))}")
# print(f"전체 문장 갯수: {len(my_corpus.sents(fileids=file_ids))}")
# print(f"어휘집 크기: {len(list(vocab))}")
# print(f"전체 토큰 갯수: {len(flattened_corpus)}")
# print(f"어휘집 샘플: {list(vocab)[:30]}")


훈련 데이터와 어휘집이 생성되었습니다.


## 8. MLE 모델 인스턴스 생성

Maximum Likelihood Estimator를 사용하여 3-gram 모델을 생성합니다.


In [39]:
# 3-gram MLE 모델 생성
lm = MLE(3)
print(f"훈련 전 어휘집 크기: {len(lm.vocab)}")
print(f"모델 order: {lm.order}")


훈련 전 어휘집 크기: 0
모델 order: 3


## 9. 모델 훈련

생성된 훈련 데이터와 어휘집으로 모델을 훈련합니다.


In [43]:
# 모델 훈련
lm.fit(train, vocab)
print("모델 훈련이 완료되었습니다!")
print(f"훈련 후 어휘집 크기: {len(lm.vocab)}")
print(f"어휘집 샘플: {list(lm.vocab)[:20]}")


모델 훈련이 완료되었습니다!
훈련 후 어휘집 크기: 6738
어휘집 샘플: ['<s>', 'The', 'Project', 'Gutenberg', 'EBook', 'of', 'Hamlet', ',', 'by', 'William', 'Shakespeare', '</s>', 'This', 'eBook', 'is', 'for', 'the', 'use', 'anyone', 'anywhere']


## 10. 텍스트 생성

훈련된 모델로 텍스트를 생성해보겠습니다. 'to be'로 시작하는 6개 단어를 생성합니다.


In [41]:
# 'to be'로 시작하는 6개 단어 생성
generated_text = lm.generate(6, ["to", "be"])
print(f"생성된 텍스트: {' '.join(generated_text)}")

# 여러 번 생성해보기
print("\n다른 생성 결과들:")
for i in range(3):
    generated = lm.generate(6, ["to", "be"])
    print(f"{i+1}: {' '.join(generated)}")


생성된 텍스트: one of their deities , and

다른 생성 결과들:
1: heard ,[ 35 ] These but
2: one man picked out of tune
3: buried in ' t ? </s>


## 11. 어휘집 lookup 테스트

어휘집에서 단어들을 찾아보고 OOV(Out-of-Vocabulary) 처리를 확인합니다.


In [42]:
# 실제 문장에서 어휘집 lookup
test_sentence = my_corpus.sents(fileids=file_ids)[1104]
lookup_result = lm.vocab.lookup(test_sentence)
print(f"원본 문장: {test_sentence}")
print(f"Lookup 결과: {lookup_result}")

# OOV 단어들 테스트
oov_words = ["aliens", "from", "Mars"]
oov_result = lm.vocab.lookup(oov_words)
print(f"\nOOV 테스트 단어들: {oov_words}")
print(f"OOV Lookup 결과: {oov_result}")
print(f"<UNK> 토큰이 사용되었습니다: {'<UNK>' in oov_result}")


원본 문장: ['_Ham', '.', '_', 'To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question', ':[', '8', ']', 'Whether', "'", 'tis', 'nobler', 'in', 'the', 'mind', 'to', 'suffer', 'The', 'slings', 'and', 'arrows', 'of', 'outrageous', 'fortune', ',', 'Or', 'to', 'take', 'arms', 'against', 'a', 'sea', 'of', 'troubles', ',[', '9', ']', 'And', ',', 'by', 'opposing', 'end', 'them', '?--', 'To', 'die', ',--', 'to', 'sleep', ',', 'No', 'more', ';--', 'and', 'by', 'a', 'sleep', ',', 'to', 'say', 'we', 'end', 'The', 'heart', '-', 'ache', ',', 'and', 'the', 'thousand', 'natural', 'shocks', 'That', 'flesh', 'is', 'heir', 'to', ':', "'", 'tis', 'a', 'consummation', 'Devoutly', 'to', 'be', 'wished', '.']
Lookup 결과: ('_Ham', '.', '_', 'To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question', ':[', '8', ']', 'Whether', "'", 'tis', 'nobler', 'in', 'the', 'mind', 'to', 'suffer', 'The', 'slings', 'and', 'arrows', 'of', 'outrageous', 'fortune', ',', 'Or', 'to', 'take', '

## 12. N-gram 빈도수 확인

모델에서 특정 N-gram의 빈도수를 확인해보겠습니다.


In [54]:
# 전체 카운트 정보
print(f"카운트 객체 타입: {type(lm.counts)}")

# 'to be'의 빈도수
to_be_count = lm.counts[["to"]]["be"]
print(f"'to be'의 빈도수: {to_be_count}")

# 'to'의 전체 빈도수
to_count = lm.counts['to']
print(f"'to'의 전체 빈도수: {to_count}")


카운트 객체 타입: <class 'nltk.lm.counter.NgramCounter'>
'to be'의 빈도수: 43
'to'의 전체 빈도수: 754


## 13. 확률 계산

단어의 확률을 계산해보겠습니다. 조건부 확률도 확인해보겠습니다.


In [55]:
# 단순 확률
be_prob = lm.score("be")
print(f"P(be) = {be_prob:.6f}")

# 조건부 확률: P(be|to)
be_given_to = lm.score("be", ["to"])
print(f"P(be|to) = {be_given_to:.6f}")

# 조건부 확률: P(be|not, to)
be_given_not_to = lm.score("be", ["not", "to"])
print(f"P(be|not, to) = {be_given_not_to:.6f}")


P(be) = 0.003210
P(be|to) = 0.057029
P(be|not, to) = 0.272727


## 14. 로그 확률 계산

매우 작은 확률값을 다루기 위해 로그 스케일로 확률을 계산합니다.


In [56]:
# 로그 확률
be_logprob = lm.logscore("be")
print(f"log P(be) = {be_logprob:.6f}")

# 조건부 로그 확률
be_given_to_log = lm.logscore("be", ["to"])
print(f"log P(be|to) = {be_given_to_log:.6f}")

be_given_not_to_log = lm.logscore("be", ["not", "to"])
print(f"log P(be|not, to) = {be_given_not_to_log:.6f}")


log P(be) = -8.283356
log P(be|to) = -4.132156
log P(be|not, to) = -1.874469


## 15. 엔트로피와 퍼플렉서티 계산

모델의 성능을 평가하기 위해 엔트로피와 퍼플렉서티를 계산합니다.


In [57]:
# 테스트 데이터
test = [("to", "be"), ("or", "not"), ("to", "be")]
print(f"테스트 데이터: {test}")

# 엔트로피 계산
entropy = lm.entropy(test)
print(f"엔트로피: {entropy:.6f}")

# 퍼플렉서티 계산
perplexity = lm.perplexity(test)
print(f"퍼플렉서티: {perplexity:.6f}")

print("\n💡 참고: 낮은 퍼플렉서티는 더 좋은 모델을 의미합니다!")


테스트 데이터: [('to', 'be'), ('or', 'not'), ('to', 'be')]
엔트로피: 4.995137
퍼플렉서티: 31.892318

💡 참고: 낮은 퍼플렉서티는 더 좋은 모델을 의미합니다!


## 결론

이 노트북에서 우리는:
1. NLTK를 사용하여 텍스트 코퍼스를 처리했습니다
2. 패딩과 N-gram 생성을 배웠습니다
3. Maximum Likelihood Estimator로 3-gram 모델을 훈련했습니다
4. 텍스트 생성을 수행했습니다
5. 확률 계산과 모델 평가 방법을 배웠습니다

N-gram 모델은 간단하지만 언어 모델링의 기초를 이해하는 데 매우 유용합니다!
